In [1]:
import datetime
import numpy as np
import pandas as pd

lost = 'Lost, Cancelled - 0%'
won = 'Win - 100%'

file_loc = r"C:\Users\Zach_Schulz-Behrend\OneDrive - Dell Technologies\Documents\data\order-oppty_analysis\SFDC pipeline\ZKSB13 - All opportunities-2025-02-25-10-23-06.csv"
file_loc_my_accts = r"c:\Users\Zach_Schulz-Behrend\OneDrive - Dell Technologies\Documents\data\account-alignments\FY26\_myAccounts.xlsx"


def rm_wins_loss(df, state=0):
    # df_oppty_wins = oppty[oppty["Stage"]==won]
    # df_oppty_loses = oppty[oppty["Stage"]==lost]
    # df_oppty_open = oppty[(oppty["Stage"]!=lost) & (oppty["Stage"]!=won)]
    
    if state not in [0,1,2]:
        return "Not valid."
    else:
        if state==0: #rm both
            df = df[df['Stage']!=lost]
            df = df[df['Stage']!=won]
        elif state==1: #rm lost
            df = df[df['Stage']!=lost]
        elif state==2: #rm won
            df = df[df['Stage']!=won]
    
    return df

def revenue_format(df):
    formatted_cols = df['Unweighted Rev'].apply(lambda x: f"${x:,.2f}")
    df['Unweighted Rev Formatted'] = formatted_cols
    cols = df.columns.to_list()
    cols[-1], cols[5] = cols[5], cols[-1]

    return df[cols].sort_values(by=["Unweighted Rev"], ascending=False)

def get_date ():
    curr_date = datetime.datetime.now()

    str_year = str(curr_date.year)

    if curr_date.month < 10:
        str_month = '0' + str(curr_date.month)
    else:
        str_month = str(curr_date.month)

    if curr_date.day < 10:
        str_day = '0' + str(curr_date.day)
    else:
        str_day = str(curr_date.day)


    return str_year + str_month + str_day

date = get_date()
file_loc_store_designs = r"c:\Users\Zach_Schulz-Behrend\OneDrive - Dell Technologies\Documents\data\order-oppty_analysis\Designs_{0}.xlsx".format(date)

# Instantiate DataFrames 

Make sure files are closed. Automate?

## Each DataFrame filtered only FY26'1H Accounts
     1. All opportunities (`oppty`)
     2. All Designs (`df_designs`)
     3. All non-Designs (`df_not_designs`)

In [2]:
oppty = pd.read_csv(file_loc)
oppty = rm_wins_loss(df=oppty)

df_my_accts = pd.read_excel(file_loc_my_accts, sheet_name="Account Search Result", nrows=68, usecols="A:B, D:G")
df_my_accts = df_my_accts.set_index(df_my_accts.ID)
df_my_accts = df_my_accts.drop("ID", axis=1)

flag_my_accts = oppty['Affinity ID'].apply(lambda x: x in df_my_accts.index)
oppty = oppty[flag_my_accts]

In [3]:
df_designs = oppty[oppty["Opportunity Type"]=="Design"]
df_not_designs = oppty[oppty["Opportunity Type"]!="Design"]

In [4]:
df_designs = df_designs.set_index("Affinity ID")

In [5]:
df_designs = revenue_format(df_designs)

In [6]:
df_designs.to_excel(file_loc_store_designs, index=False)

In [7]:
df_my_accts.head()

,Name,SE,AE,ISR,Region
ID,,,,,
2516107635,AB Sciex Canada - OEM,Davis,Shaw,Calcote,Central
3533189262,AB SCIEX LLC - OEM,Davis,Shaw,Calcote,Central
3536634246,Alert Innovation Inc - OEM,Davis,Shaw,Calcote,Central
3513936280,Artisight - OEM,Davis,Shaw,Calcote,Central
3520515806,Artisight - ST - OEM,Davis,Shaw,Calcote,Central


In [8]:
# df = df_designs.iloc[:10, :]

In [9]:
# df

In [10]:
# test_series = df['Affinity ID']

In [11]:
# df_my_accts[df_my_accts.index==2897410262].loc[2897410262,'SE':'Region']

In [17]:
index_cp = df_designs.index.to_series()

In [18]:
test = index_cp.apply(lambda x: df_my_accts[df_my_accts.index==x].loc[x,'SE':'Region'])

In [22]:
test.head(2)

,SE,AE,ISR,Region
Affinity ID,,,,
2897410262,Davis,Shaw,Turiello,Central
3606575311,Trapp,Holleran,Green,Central


In [23]:
df_designs.head(2)

,Account Name,Opportunity Owner,Opportunity Name,Stage,Unweighted Rev Currency,Unweighted Rev Formatted,Weighted Rev Currency,Weighted Rev,Probability (%),Age,Book Date,Created Date,Next Step,Opportunity Type,Unweighted Rev
Affinity ID,,,,,,,,,,,,,,,
2897410262,General Electric - G500 - OEM,Peter Seesselberg,General Electric - G500 - OEM - GEHC - MRI XR7...,Commit - 90%,USD,"$51,320,000.00",USD,15396000.0,30,701.0,5/30/2025,3/27/2023,"2/20/2025 - Derek D: Hobey got both xr7620's, ...",Design,51320000.0
3606575311,TENSTORRENT - OEM,Kendall Green,TENSTORRENT - OEM - PE R760xa Blackhole AI app...,Discover - 10%,USD,"$40,000,000.00",USD,4000000.0,10,105.0,7/31/2025,11/12/2024,1/27/2025 - Derek D: had a systems mgmt/iDRAC ...,Design,40000000.0


In [26]:
pd.merge(test, df_designs, on='Affinity ID', how='outer')

,SE,AE,ISR,Region,Account Name,Opportunity Owner,Opportunity Name,Stage,Unweighted Rev Currency,Unweighted Rev Formatted,Weighted Rev Currency,Weighted Rev,Probability (%),Age,Book Date,Created Date,Next Step,Opportunity Type,Unweighted Rev
Affinity ID,,,,,,,,,,,,,,,,,,,
2516101695,Davis,Shaw,Calcote,Central,McKesson - OEM,Derek Davis,McKesson - OEM - Pharmacy Automation STORAGE! ...,Qualify - 30%,USD,"$6,000,000.00",USD,1800000.00,30,28.0,6/30/2025,1/28/2025,"2/19/2025 - Derek D: T&B request submitted, ap...",Design,6000000.0
2516110628,Davis,Shaw,Calcote,Central,Eaton Corporation - OEM,Derek Davis,Eaton Corporation - OEM - BrightLayer Factory ...,Discover - 10%,USD,"$5,000,000.00",USD,500000.00,10,175.0,8/29/2025,9/3/2024,2/20/2025 - Derek D: POC for south Milwaukee i...,Design,5000000.0
2627674388,Davis,Shaw,Calcote,Central,Endace - OEM,Derek Davis,Endace - OEM - Endaceprobe 4100 Series - R660X...,Propose - 60%,USD,"$14,362,500.00",USD,8617500.00,60,334.0,5/30/2025,3/28/2024,2/20/2025 - Derek D: this is still moving forw...,Design,14362500.0
2627674388,Davis,Shaw,Calcote,Central,Endace - OEM,Derek Davis,Endace - OEM - EndaceProbe 2100 Series - PE XR...,Discover - 10%,USD,"$14,000,000.00",USD,1400000.00,10,200.0,6/30/2025,8/9/2024,2/10/2025 - Derek D: submitting intake for ext...,Design,14000000.0
2627674388,Davis,Shaw,Calcote,Central,Endace - OEM,Kathleen Bryan,Design (CompSwap)- Unicom/Endace - R660 XL,Discover - 10%,USD,"$3,456,000.00",USD,345600.00,10,484.0,2/28/2025,10/30/2023,Waiting for Analyst - OEM,Design,3456000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3606575311,Trapp,Holleran,Green,Central,TENSTORRENT - OEM,Kendall Green,TENSTORRENT - OEM - LoudBox - Precision T7960X...,Discover - 10%,USD,"$8,000,000.00",USD,800000.00,10,104.0,7/31/2025,11/13/2024,1/27/2025 - Derek D: had a systems mgmt/iDRAC ...,Design,8000000.0
3606575311,Trapp,Holleran,Green,Central,TENSTORRENT - OEM,Kendall Green,TENSTORRENT - OEM - PE R760xa Blackhole AI app...,Discover - 10%,USD,"$40,000,000.00",USD,4000000.00,10,105.0,7/31/2025,11/12/2024,1/27/2025 - Derek D: had a systems mgmt/iDRAC ...,Design,40000000.0
3606575311,Trapp,Holleran,Green,Central,TENSTORRENT - OEM,Kendall Green,TENSTORRENT - OEM - PE XE7745 Blackhole AI App...,Discover - 10%,USD,"$25,000,000.00",USD,2500000.00,10,97.0,6/30/2025,11/20/2024,1/16/2025 - Derek D: this is back on the table...,Design,25000000.0
